In [45]:
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('~')
from analysts_tools.growth import *
    #Procurement tools
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

In [99]:
sells_date_inicio = (datetime.datetime.today() - datetime.timedelta(days=10)).strftime('%Y-%m-%d') #1 months
sells_date_fin = (datetime.datetime.today() - datetime.timedelta(days=9)).strftime('%Y-%m-%d') #1 months
print(sells_date_inicio,sells_date_fin)

2023-10-31 2023-11-01


In [47]:
query = """
SELECT 
    s.identifier_value AS region_code,
    su.card_id,
    AVG(fbi.margin_mtd) AS margin
FROM dpr_cross_business.fact_cross_business_insights fbi
INNER JOIN dpr_shared.dim_stock_unit su ON su.stock_unit_id = fbi.dim_stock_unit
INNER JOIN dpr_shared.dim_site s ON s.site_id = fbi.dim_site
WHERE (dim_date,dim_stock_unit) IN (SELECT MAX(dim_date), dim_stock_unit FROM dpr_cross_business.fact_cross_business_insights WHERE margin_mtd <> 0 GROUP BY 2)
GROUP BY 1,2
"""
df_margen = run_read_dwd_query(query)
df_margen[["margin"]] = df_margen[["margin"]].astype(float)

In [100]:
file1 = pd.read_csv('SH_GRADS2023-10-31SPOGROUPS_9a16.csv')
file2 = pd.read_csv('SH_GRADS2023-10-31BOGGROUPS_9a16.csv')
file3 = pd.read_csv('SH_GRADS2023-10-31MDEGROUPS_9a16.csv')
file4 = pd.read_csv('SH_GRADS2023-10-31CMXGROUPS_9a16.csv')
file5 = pd.read_csv('SH_GRADS2023-10-31VCPGROUPS_9a16.csv')
file6 = pd.read_csv('SH_GRADS2023-10-31BHZGROUPS_9a16.csv')
file7 = pd.read_csv('SH_GRADS2023-10-31CWBGROUPS_9a16.csv')
file8 = pd.read_csv('SH_GRADS2023-10-31BAQGROUPS_9a16.csv')

df_groups = pd.concat([file1, 
                       file2, 
                       file3, 
                       file4, 
                       file5, 
                       file6, 
                       file7,
                       file8
                       ], ignore_index=True) #, 

In [101]:
df_groups.groupby(['city','group']).customer_id.count().reset_index()

,city,group,customer_id
0,BAQ,control,944
1,BAQ,test,944
2,BHZ,control,1206
3,BHZ,test,1207
4,BOG,control,2775
5,BOG,test,2776
6,CMX,control,2452
7,CMX,test,2452
8,CWB,control,612
9,CWB,test,613


In [102]:
df_groups_info = df_groups.groupby(['group']).size().reset_index()
df_groups_info = df_groups_info.rename(columns={0: 'total_users'})

In [103]:
df_groups_info

,group,total_users
0,control,13834
1,test,13839


In [104]:
query = """
SELECT 
    s.identifier_value AS region_code,
    DATE(fs.order_submitted_date) AS submit_date,
    --fs.order_close_date AS close_date,
    --fs.order_id,
    dc.source_id AS customer_id,
    --cat.parent_description as category,
    --cat.description AS subcategory,
    dp.card_id,
    --dp.source_id AS sku_id,
    dp.card_description AS product_name,
    --fs.product_quantity_x_step_unit AS cant,
    CASE
        WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fs.gmv_pxq_local/3776
        WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fs.gmv_pxq_local/4.75
        WHEN s.identifier_value IN ('CMX') THEN fs.gmv_pxq_local/19.65
    ELSE fs.gmv_pxq_local END AS gmv_usd,
    CASE
        WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fsd.product_discount/3776
        WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fsd.product_discount/4.75
        WHEN s.identifier_value IN ('CMX') THEN fsd.product_discount/19.65
    ELSE fsd.product_discount END AS discount_applied,
    CASE
        WHEN (fsd.adjustment_reason ILIKE 'SH\_GRAD%' OR fsd.adjustment_reason ILIKE 'EXTRA\_INV\_GRAD%') THEN 'EXP'
        WHEN fsd.adjustment_reason IS NULL THEN 'NO DISCOUNT'
    ELSE 'OTHERS DCT' END AS responsable,
    fsd.adjustment_reason--,
    --bo.source_id AS offer_id
    
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
LEFT JOIN dpr_sales.dim_offer               bo  ON bo.offer_id = fsd.dim_offer     

WHERE 
    fs.gmv_enabled = TRUE
    AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
    AND fs.fb_order_status_id  IN (1,6,7,8)
    AND fs.is_deleted = FALSE
    AND fs.dim_status = 1
    AND dp.is_slot = 'false'
    AND DATE(fs.order_submitted_date) between '{Fecha_start}' AND '{Fecha_end}'
    --AND (fsd.adjustment_reason ILIKE 'SH\_GRAD%' OR fsd.adjustment_reason ILIKE 'SH\_FARM%')
    AND dc.source_id IN {clientes}
""".format(Fecha_start = sells_date_inicio, Fecha_end = sells_date_fin, clientes = tuple(set(df_groups.customer_id.unique())))

df = run_read_dwd_query(query)
df[["gmv_usd", "discount_applied"]] = df[["gmv_usd", "discount_applied"]].astype('float32')

df['discount_applied'].fillna(0, inplace=True)

df = pd.merge(df, df_groups[['customer_id','group']], left_on='customer_id', right_on='customer_id', how='inner')
df = pd.merge(df, df_margen[['margin','card_id']], left_on='card_id', right_on='card_id', how='inner')

df["cash_margin"] = df.gmv_usd * df.margin
df["net_cash_margin"] = df.cash_margin - df.discount_applied

In [105]:
df[['customer_id','group']].drop_duplicates().group.value_counts()

test       1558
control    1389
Name: group, dtype: int64

In [106]:
df2 = df#.loc[df.region_code == 'VCP'].copy()

In [107]:
results_offer = (df2.groupby(by=['region_code', 'group'])
                              .agg(customer_count=('customer_id', 'nunique'),
                                   gmv_usd=('gmv_usd', np.sum),
                                   discount_usd=('discount_applied', np.sum),
                                   cash_m=('cash_margin', np.sum),
                                   net_cash_m=('net_cash_margin',np.sum)
                              ).reset_index()
               )

In [108]:
df3 =  pd.merge(results_offer, df_groups.groupby(['city','group']).customer_id.count().reset_index(), left_on=['region_code','group'], right_on=['city','group'], how='inner')

# Drop 'city' and rename 'customer_id' to 'all_customers'
df_updated = df3.drop('city', axis=1).rename(columns={'customer_id': 'all_customers'})

# Calculate conversion rate
df_updated['conversion_rate'] = 100.0 * df_updated['customer_count'] / df_updated['all_customers']

In [109]:
df_updated

,region_code,group,customer_count,gmv_usd,discount_usd,cash_m,net_cash_m,all_customers,conversion_rate
0,BAQ,control,80,4650.500977,159.842270,522.931495,363.089219,944,8.474576
1,BAQ,test,86,6035.283203,282.428680,704.585351,422.156666,944,9.110169
2,BHZ,control,98,9868.844727,553.926147,1440.610519,886.684368,1206,8.126036
3,BHZ,test,106,9073.160156,576.953491,1254.996695,678.043180,1207,8.782104
4,BOG,control,272,20176.375000,587.303833,2358.151870,1770.848023,2775,9.801802
5,BOG,test,297,23090.281250,1139.439819,2994.375567,1854.935806,2776,10.698847
6,CMX,control,365,29645.509766,1141.265259,3797.058903,2655.793670,2452,14.885808
7,CMX,test,401,35740.007812,2098.806396,4963.696398,2864.889957,2452,16.353997
8,CWB,control,53,8076.898926,536.892578,475.005709,-61.886861,612,8.660131
9,CWB,test,60,7259.940918,453.334656,566.123681,112.789035,613,9.787928
